#  PROPHET

In [3]:
%%time                   
import pandas as pd         # прогноз по SKU без сезенности(Более подходящий) АГРОКОМПЛЕКС и ТД....                  
from prophet import Prophet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/SKU for prophet.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Код товара': 'SKU', 'Продажи, кг': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по товару
grouped_data = data.groupby('SKU')

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждого товара
for sku, sku_data in grouped_data:
    # Проверка на наличие пропущенных значений в столбце 'y'
    if sku_data['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для товаров с меньше чем 2 непропущенными значениями 'y'

    # Создание и обучение модели Prophet для каждого товара
    model = Prophet()
    model.fit(sku_data[['ds', 'y']])

    # Создание DataFrame для прогноза на 60 дней вперед
    future = model.make_future_dataframe(periods=90)
    forecast = model.predict(future)
    forecast['SKU'] = sku  # Добавление информации о товаре

    # Замена отрицательных значений прогноза на 0
    forecast['yhat'] = forecast['yhat'].where(forecast['yhat'] > 0, 0)

    # Добавление прогноза для текущего товара к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'SKU', 'yhat', 'yhat_upper']
all_forecasts[selected_columns].to_excel('прогноз_продаж_по_SKU.xlsx', index=False)



16:04:46 - cmdstanpy - INFO - Chain [1] start processing
16:04:46 - cmdstanpy - INFO - Chain [1] done processing
16:04:46 - cmdstanpy - INFO - Chain [1] start processing
16:04:46 - cmdstanpy - INFO - Chain [1] done processing
16:04:47 - cmdstanpy - INFO - Chain [1] start processing
16:04:47 - cmdstanpy - INFO - Chain [1] done processing
16:04:47 - cmdstanpy - INFO - Chain [1] start processing
16:04:47 - cmdstanpy - INFO - Chain [1] done processing
16:04:47 - cmdstanpy - INFO - Chain [1] start processing
16:04:47 - cmdstanpy - INFO - Chain [1] done processing
16:04:47 - cmdstanpy - INFO - Chain [1] start processing
16:04:47 - cmdstanpy - INFO - Chain [1] done processing
16:04:47 - cmdstanpy - INFO - Chain [1] start processing
16:04:48 - cmdstanpy - INFO - Chain [1] done processing
16:04:48 - cmdstanpy - INFO - Chain [1] start processing
16:04:48 - cmdstanpy - INFO - Chain [1] done processing
16:04:48 - cmdstanpy - INFO - Chain [1] start processing
16:04:48 - cmdstanpy - INFO - Chain [1]

CPU times: total: 17.9 s
Wall time: 31.3 s


In [ ]:
%%time  
                                       # прогноз МОТИВАЦИЯ МАГАЗИНОВ
import pandas as pd
from prophet import Prophet

# Загрузка данных из Excel-файла с корректным форматом даты
data = pd.read_excel('C:/Users/bondarenKovv/Desktop/Python/Прогноз SKU/Продажи для мотивации маг.xlsx')

# Преобразование формата даты
data['Дата'] = pd.to_datetime(data['Дата'], format='%d.%m.%Y', dayfirst=True)

# Подготовка данных для применения модели Prophet
data = data.rename(columns={'Дата': 'ds', 'Магазин': 'store', 'Класс2': 'class', 'Продажи': 'y'})
data['y'] = data['y'].astype(float)

# Группировка данных по магазину и Класс2
grouped_data = data.groupby(['store', 'class'])

# Создание пустого DataFrame для хранения прогнозов
all_forecasts = pd.DataFrame()

# Прогнозирование продаж для каждой комбинации магазина и Класс2
for (store, class2), group in grouped_data:
    # Проверка на наличие пропущенных значений в столбце 'y'
    if group['y'].notnull().sum() < 2:
        continue  # Пропускаем прогноз для групп с менее чем 2 непропущенными значениями 'y'

    # Создание и обучение модели Prophet для каждой комбинации магазина и Класс2
    model = Prophet()
    model.fit(group[['ds', 'y']])

    # Создание DataFrame для прогноза на 30 дней вперед
    future = model.make_future_dataframe(periods=35)
    forecast = model.predict(future)
    forecast.loc[forecast['yhat'] < 0, 'yhat'] = 0
    forecast['store'] = store  # Добавление информации о магазине
    forecast['class'] = class2  # Добавление информации о классе

    # Добавление прогноза для текущей комбинации магазина и Класс2 к общему DataFrame
    all_forecasts = pd.concat([all_forecasts, forecast])

# Сохранение результатов в новом Excel-файле
selected_columns = ['ds', 'store', 'class', 'yhat', 'yhat_lower', 'yhat_upper']
all_forecasts[selected_columns].to_excel('прогноз_продаж_мотивация_магазинов.xlsx', index=False)
